In [1]:
import os
import polars as pl
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras import layers, models, optimizers, regularizers, callbacks


In [20]:
def get_generator_v3(dataframe, weights, feature_names, label_name, shuffle=True, batch_size=8192):
    def generator():
        indices = np.arange(len(dataframe))
        if shuffle:
            np.random.shuffle(indices)

        num_batches = len(indices) // batch_size + (1 if len(indices) % batch_size > 0 else 0)

        for i in range(num_batches):
            start_index = i * batch_size
            end_index = min((i + 1) * batch_size, len(dataframe))
            current_indices = indices[start_index:end_index]

            features = dataframe.iloc[current_indices][feature_names].values
            labels = dataframe.iloc[current_indices][label_name].values
            if weights is not None:
                weights_batch = weights.iloc[current_indices].values
            else:
                weights_batch = np.ones(len(labels), dtype=np.float32)

            yield features, labels.reshape(-1, 1), weights_batch

    return generator


def prepare_dataset(dataframe, weights, feature_names, label_name, batch_size=8192, shuffle=True):
    num_features = len(feature_names)

    output_signature = (
        tf.TensorSpec(shape=(None, num_features), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )

    dataset = tf.data.Dataset.from_generator(
        get_generator_v3(dataframe, weights, feature_names, label_name, shuffle, batch_size),
        output_signature=output_signature
    )

    # dataset = dataset.repeat()

    return dataset

In [3]:
def create_model(input_dim, lr, weight_decay):
    # Create a sequential model
    model = models.Sequential()

    # Start with the first hidden layer with BatchNorm1d, SiLU, Dropout, and Linear layers
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Add BatchNorm, SiLU (Swish in TensorFlow), Dropout, and Dense (Linear) layers
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))

    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(weight_decay)))


    model.add(layers.BatchNormalization())
    model.add(layers.Activation('swish'))
    model.add(layers.Dropout(0.1))  # Assuming dropouts[1] is valid
    model.add(layers.Dense(128, kernel_regularizer=regularizers.l2(weight_decay)))


    # Output layer
    model.add(layers.Dense(1, activation='tanh'))

    # Compile model with Mean Squared Error loss
    # model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='mse', metrics=[WeightedR2()])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr),loss='mse',metrics=[tf.keras.metrics.R2Score(class_aggregation='uniform_average')])
    return model

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
feature_names = [f"feature_{i:02d}" for i in range(79)] + [f"feature_{i:02d}_lag_1" for i in range(79)] + [f"responder_{idx}_lag_1" for idx in range(9)]
label_name = 'responder_6'
weight_name = 'weight'


### Get Train and Valid data

In [6]:
start_id = 1300
end_id = 1664
folder_paths = [
    f"/content/drive/MyDrive/JaneSt/CustomData3_featurelags/training.parquet/date_id={date_id}/*.parquet"
    for date_id in range(start_id, end_id + 1)
]
lazy_frames = [pl.scan_parquet(path) for path in folder_paths]
combined_lazy_df = pl.concat(lazy_frames)


In [7]:
df = combined_lazy_df.collect().to_pandas()

In [8]:
# df = pl.scan_parquet(f"/content/drive/MyDrive/JaneSt/CustomData3_featurelags/training.parquet").collect().to_pandas()
valid = pl.scan_parquet(f"/content/drive/MyDrive/JaneSt/CustomData3_featurelags/validation.parquet").collect().to_pandas()

In [9]:
# df = pd.concat([df, valid]).reset_index(drop=True)
df[feature_names] = df[feature_names].ffill().fillna(0)
valid[feature_names] = valid[feature_names].ffill().fillna(0)

In [10]:

X_train = df[ feature_names ]
y_train = df[ label_name ]
w_train = df[ "weight" ]
X_valid = valid[ feature_names ]
y_valid = valid[ label_name ]
w_valid = valid[ "weight" ]


In [21]:

train_dataset = prepare_dataset(df, w_train, feature_names, label_name, batch_size=8129)
valid_dataset = prepare_dataset(valid, w_valid, feature_names, label_name, batch_size=8129)


In [12]:
print(f"max epochs can train is: {len(X_train) / 8192}")

max epochs can train is: 1635.9814453125


In [13]:
max_epochs = int(len(X_train) / 8192)

In [15]:
lr = 0.01
weight_decay = 5e-4

input_dim = df[feature_names].shape[1]
model = create_model(input_dim=input_dim, lr = lr, weight_decay=weight_decay)
model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization_3                │ (None, 167)                 │             668 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 167)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 167)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │          86,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_5 (Activation)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 419,229 (1.60 MB)

 Trainable params: 416,847 (1.59 MB)

 Non-trainable params: 2,382 (9.30 KB)

In [18]:
ca = [
    tf.keras.callbacks.EarlyStopping(monitor='val_r2_score', patience=50, mode='max'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=f'/content/drive/MyDrive/JaneSt/models/tf_nn_model13_morelagfeatures.keras',
        monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_r2_score',  # Metric to be monitored
        factor=0.1,  # Factor by which the learning rate will be reduced
        patience=20,  # Number of epochs with no improvement after which learning rate will be reduced
        verbose=1,  # Verbosity mode
        min_lr=1e-6  # Lower bound on the learning rate
    )
]


In [ ]:
ca = [
    tf.keras.callbacks.EarlyStopping(monitor='val_r2_score', patience=50, mode='max'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/MyDrive/JaneSt/models/tf_nn_model13_morelagfeatures_epoch{epoch:02d}.keras',
        monitor='val_loss',
        save_best_only=False,  # Set to False to save every epoch
        save_freq='epoch'  # Save at the end of each epoch
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_r2_score',
        factor=0.1,
        patience=20,
        verbose=1,
        min_lr=1e-6
    )
]

In [22]:
model.fit(
    train_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    epochs=200,
    validation_data=valid_dataset.map(lambda x, y, w: (x, y, {'sample_weight': w})),
    callbacks=ca
)

Epoch 1/200
   1649/Unknown 143s 86ms/step - loss: 1.6492 - r2_score: 0.0114

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1649/1649 ━━━━━━━━━━━━━━━━━━━━ 158s 95ms/step - loss: 1.6492 - r2_score: 0.0114 - val_loss: 1.1327 - val_r2_score: 5.8770e-05 - learning_rate: 0.0100
Epoch 2/200


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1649/1649 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 1.6476 - r2_score: 0.0117

...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1649/1649 ━━━━━━━━━━━━━━━━━━━━ 150s 90ms/step - loss: 1.6238 - r2_score: 0.0223 - val_loss: 1.1195 - val_r2_score: 0.0037 - learning_rate: 1.0000e-04
Epoch 61/200


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


  56/1649 ━━━━━━━━━━━━━━━━━━━━ 2:17 86ms/step - loss: 1.6453 - r2_score: 0.0224

KeyboardInterrupt: 

In [24]:
model

<Sequential name=sequential_1, built=True>

In [26]:
# This should be a NumPy array or a Tensor with shape (num_samples, num_features)
X_new = X_valid.to_numpy()  # Replace with actual data

# Make predictions
predictions = model.predict(X_new)
r2_metric = tf.keras.metrics.R2Score(class_aggregation='uniform_average')

if not isinstance(y_valid, np.ndarray):
    y_valid = y_valid.to_numpy()  # Convert to numpy array if it is a DataFrame

r2_metric.update_state(y_true=y_valid, y_pred=predictions)
r2_score_value = r2_metric.result().numpy()

print(f"R² Score on validation data: {r2_score_value}")

39598/39598 ━━━━━━━━━━━━━━━━━━━━ 49s 1ms/step
R² Score on validation data: 0.0037530064582824707


In [25]:
model.save('/content/drive/MyDrive/JaneSt/models/tf_nn_model13.keras')

In [28]:
model.save_weights('/content/drive/MyDrive/JaneSt/models/tf_nn_model13.weights.h5')

## Resume Training